# Problem 212: Combined Volume of Cuboids

An <dfn>axis-aligned cuboid</dfn>, specified by parameters $\{(x_0, y_0, z_0), (dx, dy, dz)\}$, consists of all points $(X,Y,Z)$ such that $x_0 \le X \le x_0 + dx$, $y_0 \le Y \le y_0 + dy$ and $z_0 \le Z \le z_0 + dz$.  The volume of the cuboid is the product, $dx \times dy \times dz$.  The <dfn>combined volume</dfn> of a collection of cuboids is the volume of their union and will be less than the sum of the individual volumes if any cuboids overlap.

Let $C_1, \dots, C_{50000}$ be a collection of $50000$ axis-aligned cuboids such that $C_n$ has parameters

\begin{align}
x_0 &= S_{6n - 5} \bmod 10000 \nonumber \\
y_0 &= S_{6n - 4} \bmod 10000 \nonumber \\
z_0 &= S_{6n - 3} \bmod 10000 \nonumber \\
dx &= 1 + (S_{6n - 2} \bmod 399) \nonumber \\
dy &= 1 + (S_{6n - 1} \bmod 399) \nonumber \\
dz &= 1 + (S_{6n} \bmod 399) \nonumber
\end{align}

where $S_1,\dots,S_{300000}$ come from the "Lagged Fibonacci Generator":

For $1 \le k \le 55$, $S_k = [100003 - 200003k + 300007k^3] \pmod{1000000}$. \
For $56 \le k$, $S_k = [S_{k -24} + S_{k - 55}] \pmod{1000000}$.

Thus, $C_1$ has parameters $\{(7,53,183),(94,369,56)\}$, $C_2$ has parameters $\{(2383,3563,5079),(42,212,344)\}$, and so on.

The combined volume of the first $100$ cuboids, $C_1, \dots, C_{100}$, is $723581599$.

What is the combined volume of all $50000$ cuboids, $C_1, \dots, C_{50000}$?

In [1]:
from functools import cache


@cache
def S(k: int) -> int:
    if k <= 55:
        return (100003 - 200003 * k + 300007 * k * k * k) % 1000000
    else:
        return (S(k - 24) + S(k - 55)) % 1000000


def C(n: int):
    x0 = S(6 * n - 5) % 10000
    y0 = S(6 * n - 4) % 10000
    z0 = S(6 * n - 3) % 10000
    dx = 1 + (S(6 * n - 2) % 399)
    dy = 1 + (S(6 * n - 1) % 399)
    dz = 1 + (S(6 * n) % 399)
    return [x0, y0, z0, dx, dy, dz]


S.cache_clear()
print("C(1) = ", C(1))
print("C(2) = ", C(2))

C(1) =  [7, 53, 183, 94, 369, 56]
C(2) =  [2383, 3563, 5079, 42, 212, 344]


In [2]:
from dataclasses import dataclass
from functools import cached_property
from typing import Tuple


@dataclass
class Cuboid:
    x0: int
    y0: int
    z0: int
    dx: int
    dy: int
    dz: int

    def __post_init__(self):
        self.overlapping_cuboids = []

    @cached_property
    def volume(self) -> int:
        return self.dx * self.dy * self.dz

    @property
    def xmin(self) -> int:
        return self.x0

    @property
    def ymin(self) -> int:
        return self.y0

    @property
    def zmin(self) -> int:
        return self.z0

    @cached_property
    def xmax(self) -> int:
        return self.x0 + self.dx

    @cached_property
    def ymax(self) -> int:
        return self.y0 + self.dy

    @cached_property
    def zmax(self) -> int:
        return self.z0 + self.dz

    def x_overlap(self, other: "Cuboid") -> Tuple[int, int]:
        xmin = max(self.xmin, other.xmin)
        xmax = min(self.xmax, other.xmax)
        dx = xmax - xmin
        return (xmin, dx)

    def y_overlap(self, other: "Cuboid") -> Tuple[int, int]:
        ymin = max(self.ymin, other.ymin)
        ymax = min(self.ymax, other.ymax)
        dy = ymax - ymin
        return (ymin, dy)

    def z_overlap(self, other: "Cuboid") -> Tuple[int, int]:
        zmin = max(self.zmin, other.zmin)
        zmax = min(self.zmax, other.zmax)
        dz = zmax - zmin
        return (zmin, dz)

    def overlaps(self, other: "Cuboid") -> bool:
        if self.xmin >= other.xmax:
            return False
        elif self.xmax <= other.xmin:
            return False
        if self.ymin >= other.ymax:
            return False
        elif self.ymax <= other.ymin:
            return False
        if self.zmin >= other.zmax:
            return False
        elif self.zmax <= other.zmin:
            return False
        else:
            return True

    def union(self, other: "Cuboid") -> "Cuboid":
        xmin, dx = self.x_overlap(other)
        ymin, dy = self.y_overlap(other)
        zmin, dz = self.z_overlap(other)
        if dx <= 0 or dy <= 0 or dz <= 0:
            return Cuboid(0, 0, 0, 0, 0, 0)
        return Cuboid(xmin, ymin, zmin, dx, dy, dz)


c1 = Cuboid(0, 0, 0, 10, 10, 10)
c2 = Cuboid(10, 0, 0, 10, 10, 10)
c3 = Cuboid(4, 4, 4, 2, 2, 20)
print("c1.volume", c1.volume)
print("c1.overlaps(c2) =", c1.overlaps(c2))
print("c1.union(c2) =", c1.union(c2))
print("c1.union(c3) =", c1.union(c3))

c1.volume 1000
c1.overlaps(c2) = False
c1.union(c2) = Cuboid(x0=0, y0=0, z0=0, dx=0, dy=0, dz=0)
c1.union(c3) = Cuboid(x0=4, y0=4, z0=4, dx=2, dy=2, dz=6)


In [3]:
from functools import reduce
from itertools import combinations


n_cuboid = 50000

cuboids = [Cuboid(*C(n)) for n in range(1, n_cuboid + 1)]
# sort by zmin to allow for quicker testing of overlaps
cuboids = sorted(cuboids, key=lambda x: x.zmin)

# find overlapping cuboids
for i, c1 in enumerate(cuboids):
    for c2 in cuboids[i + 1 :]:
        if c1.zmax <= c2.zmin:
            break
        if c1.overlaps(c2):
            # overlap is only registered on c1 to avoid double counting later
            c1.overlapping_cuboids.append(c2)


combined_volume = 0
for cuboid in cuboids:
    combined_volume += cuboid.volume
    for repeats in range(1, len(cuboid.overlapping_cuboids) + 1):
        for cuboid_combinations in combinations(cuboid.overlapping_cuboids, repeats):
            cuboid_union = cuboid.union(
                reduce(lambda x, y: x.union(y), cuboid_combinations)
            )
            union_volume = cuboid_union.volume
            # inclusion exclusion principle
            combined_volume += (-1) ** (repeats) * union_volume
print("Combined volume =", combined_volume)

Combined volume = 328968937309
